# Data Wrangling for Job Postings
___


In [1]:
# Imports
import pandas as pd
import numpy as np
import re

from html.parser import HTMLParser

## Custom Functions
___

In [2]:
class CleanData:
    def __init__(self, df):
        self.df = df
        
    def remove_html_tags(self, column):
        return self.df[column].apply(lambda x: re.sub('<[^<]+?>', '', x))

    def replace_escapes(self, column):
        return self.df[column].apply(lambda x: ''.join([i for i in x.splitlines() if i != '']))

In [3]:
def make_id(data, letter):
    return [letter + str(i).rjust(4, "0") for i in range(len(data))]

## Reading and Cleaning Collected Sources
___

### Indeed Job Posting Dataset from [PromptCloud and DataStock](https://data.world/promptcloud/indeed-job-posting-dataset).
SQL Query:
```SQL
SELECT DISTINCT job_title, job_description, job_type, location, city, state, salary_from, salary_to, company_name, companydescription
FROM marketing_sample_for_trulia_com_real_estate_20190901_20191031_30k_data
WHERE (job_title LIKE '%AI%'
    OR job_title LIKE '%IT%'
    OR job_title LIKE '%Software%'
    OR job_title LIKE '%Engineer%'
    OR job_title LIKE '%Data%'
    OR job_title LIKE '%Analyst%'
    AND job_title NOT LIKE 'Air%');
```

**Preprocessing prior to merging with other job postings datasets:**
- Create Unique ID
- Convert titles and company names text to lowercase
- Create a `job_docs` column for merging with other data sets
- Create columns for merging and fill them with NAN values
- Preview job posting used in preprocessing

In [4]:
# Read in data
data_indeed = pd.read_csv('../data/postings/postings_indeed.csv', index_col=None)
data_indeed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28848 entries, 0 to 28847
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   job_title           28848 non-null  object 
 1   job_description     28848 non-null  object 
 2   job_type            0 non-null      float64
 3   location            28848 non-null  object 
 4   city                28848 non-null  object 
 5   state               28848 non-null  object 
 6   salary_from         0 non-null      float64
 7   salary_to           0 non-null      float64
 8   company_name        28846 non-null  object 
 9   companydescription  19050 non-null  object 
 10  job_docs            28848 non-null  object 
 11  id                  28848 non-null  object 
dtypes: float64(3), object(9)
memory usage: 2.6+ MB


#### Preprocess Data

In [5]:
# Create Unique ID
data_indeed['id'] = make_id(data_indeed, letter='I')

In [6]:
# Convert titles and company names text to lowercase
data_indeed['job_title'] = data_indeed['job_title'].apply(lambda x: str(x).lower())
data_indeed['company_name'] = data_indeed['company_name'].apply(lambda x: str(x).lower())

In [7]:
# Create a `job_docs` column
# separate words that were combined by subdividing at capital letters
data_indeed['job_description'] = data_indeed['job_description'].apply(lambda x: re.sub(r'[A-Z]+[a-z]+', r" \g<0> ", x))
# remove html tags
data_indeed['job_docs'] = CleanData(data_indeed).remove_html_tags(column='job_description')
# remove escape characters for things like new lines
data_indeed['job_docs'] = CleanData(data_indeed).replace_escapes(column='job_docs')

In [8]:
# Set new column values
data_indeed['category'] = np.nan
data_indeed['url'] = np.nan
data_indeed['job_board'] = np.nan

#### Preview column used in natural language processing

In [9]:
data_indeed.job_docs[0][:200]

' Ceridian .  Makes   Work   Life   Better ™ This  is our promise.  Not  only for our customers, but our employees as well. Do  you thrive in an innovative and exciting environment that’s continuously '

### Job Posts & Online Courses Study found on [Mendeley](https://data.mendeley.com/datasets/8svrmx2cmv).

**Preprocessing prior to merging with other job postings datasets:**
- Create Unique ID
- Convert titles and company names text to lowercase
- Create a `job_docs` column for merging with other data sets
- Rename `job_link` column to `url` for merging
- Create new columns for merging and fill them with NAN values
    - columns include: `company_name`, `city`, `state`
- Preview job posting used in preprocessing

In [10]:
# Read in Data
data_mendeley = pd.read_csv('../data/postings/postings_mendeley.csv', index_col=None)
data_mendeley.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9067 entries, 0 to 9066
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        9067 non-null   object
 1   job_description  9067 non-null   object
 2   job_link         9067 non-null   object
 3   job_docs         9067 non-null   object
 4   id               9067 non-null   object
dtypes: object(5)
memory usage: 354.3+ KB


#### Preprocess Data

In [11]:
# Create Unique ID
data_mendeley['id'] = make_id(data_mendeley, letter='M')

In [12]:
# Convert titles and company names text to lowercase
data_mendeley['job_title'] = data_mendeley['job_title'].apply(lambda x: str(x).lower())

In [13]:
# Create a 'job_docs' column
# separate words that were combined by subdividing at capital letters
data_mendeley['job_description'] = data_mendeley['job_description'].apply(lambda x: re.sub(r'[A-Z]+[a-z]+', r" \g<0> ", x))
# remove escape characters for things like new lines
data_mendeley['job_docs'] = CleanData(data_mendeley).replace_escapes(column='job_description')

In [14]:
# Rename 'job_link' column to 'url' for merging
data_mendeley.rename(columns={'job_link': 'url'}, inplace=True)

In [15]:
# Set new column values
data_mendeley['job_board'] = np.nan
data_mendeley['job_type'] = np.nan
data_mendeley['category'] = np.nan
data_mendeley['company_name'] = np.nan
data_mendeley['city'] = np.nan
data_mendeley['state'] = np.nan

#### Preview column used in natural language processing

In [16]:
data_mendeley.job_docs[0][:200]

" Education  Bachelor 's  Degree  Skills  SQLTime   Management  No  SQLIATHadoop  Raytheon  is supporting a U.S.  Government  customer on a large mission critical development and sustainment program wi"

### Data Scientist Job Postings found on [Kaggle](https://www.kaggle.com/datasets/jobspikr/data-scientist-job-postings-from-the-usa) from JobsPikr. 

**Preprocessing prior to merging with other job postings datasets:**
- Create Unique ID
- Convert titles and company names text to lowercase
- Create a `job_docs` column for merging with other data sets
- Preview job posting used in preprocessing

In [17]:
# Read in Data
data_jobspikr = pd.read_csv('../data/postings/postings_jobspikr.csv', index_col='crawl_timestamp')
data_jobspikr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 2019-02-06 05:26:22 +0000 to 2019-09-27 00:53:10 +0000
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   10000 non-null  object 
 1   job_title             10000 non-null  object 
 2   category              9118 non-null   object 
 3   company_name          9999 non-null   object 
 4   city                  9751 non-null   object 
 5   state                 9584 non-null   object 
 6   country               10000 non-null  object 
 7   inferred_city         8980 non-null   object 
 8   inferred_state        9484 non-null   object 
 9   inferred_country      9505 non-null   object 
 10  post_date             10000 non-null  object 
 11  job_description       10000 non-null  object 
 12  job_type              10000 non-null  object 
 13  salary_offered        597 non-null    object 
 14  job_board             9310 non-

#### Preprocess Data

In [18]:
# Create Unique ID
data_jobspikr['id'] = make_id(data_jobspikr, letter='J')

In [19]:
# Convert titles and company names text to lowercase
data_jobspikr['job_title'] = data_jobspikr['job_title'].apply(lambda x: str(x).lower())
data_jobspikr['company_name'] = data_jobspikr['company_name'].apply(lambda x: str(x).lower())

In [20]:
# Create a `job_docs` column for merging with other data sets
data_jobspikr['job_docs'] = CleanData(data_jobspikr).replace_escapes(column='job_description')

#### Preview job posting used in preprocessing

In [21]:
data_jobspikr['job_docs'][0][:200]

'Read what people are saying about working here. We are Farmers!Join a team of diverse professionals at Farmers to acquire skills on the job and apply your learned knowledge to future roles at Farmers.'

## Merging Datasets
___

In [22]:
# Select columns for merging
desired_columns = ['id','job_title', 'job_docs', 'job_type', 'category', 'job_board', 'company_name', 'city', 'state', 'url']

In [23]:
# Axis 0 merge on desired columns
data_all_postings = pd.concat([data_indeed[desired_columns], data_jobspikr[desired_columns], data_mendeley[desired_columns]])

In [24]:
# Combine job title with job description to use in NLP
data_all_postings['full_desc'] = data_all_postings['job_title'] + " " + data_all_postings['job_docs']

### Preview Merged Data

In [25]:
data_all_postings

,id,job_title,job_docs,job_type,category,job_board,company_name,city,state,url,full_desc
0,I0000,"manager, implementation",Ceridian . Makes Work Life Better ™ Th...,NaN,NaN,NaN,ceridian,Saint Petersburg,FL,NaN,"manager, implementation Ceridian . Makes W..."
1,I0001,account executive (digital sales),Salem Media Group in the Twin Cities ...,NaN,NaN,NaN,salem media group,Eagan,MN,NaN,account executive (digital sales) Salem Med...
2,I0002,"manager, sales development",We are growing! Join us to make subscripti...,NaN,NaN,NaN,recurly,San Francisco,CA,NaN,"manager, sales development We are growing! ..."
3,I0003,human resources expert,Description :ALL ABOUT TARGET As part of our...,NaN,NaN,NaN,target,Glendale,CA,NaN,human resources expert Description :ALL ABOUT...
4,I0004,cell phone technician,**LOOOKING TO HIRE WITHIN DAYS AND START ASAP*...,NaN,NaN,NaN,the fix cell phone/computer repair,Silver Spring,MD,NaN,cell phone technician **LOOOKING TO HIRE WITHI...
...,...,...,...,...,...,...,...,...,...,...,...
9062,M9062,senior business services analyst,RESPONSIBILITIES: Kforce has a client that is...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.careerbuilder.com/job/J3T26Q6VDXW9...,senior business services analyst RESPONSIBILIT...
9063,M9063,avp of corporate strategy & business strategy,"RESPONSIBILITIES: Kforce has a client, a For...",NaN,NaN,NaN,NaN,NaN,NaN,https://www.careerbuilder.com/job/J3N4Z26H1LX0...,avp of corporate strategy & business strategy ...
9064,M9064,business analyst - registered nurse,Company Overview : Creating value by bring...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.careerbuilder.com/job/JDD66078BTB8...,business analyst - registered nurse Company ...
9065,M9065,systems analyst,A leading consultancy is recruiting for a Sys...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.careerbuilder.com/job/J3Q0PN78ZKRP...,systems analyst A leading consultancy is recru...


## Export new datasets
___

In [26]:
data_all_postings.to_csv('../data/postings/postings.csv', index='id')